In [1]:
#!/usr/bin/env python3

import os, sys
import numpy as np
import itertools
import matplotlib.pyplot as plt

import scipy.linalg
import scipy.sparse

from scipy.integrate import solve_ivp
from matplotlib.lines import Line2D

from dicke_methods import coherent_spin_state, spin_op_vec_mat_dicke
from correlator_methods import compute_correlators, vec_zxy_to_pzm, multiply_vecs

np.set_printoptions(linewidth = 200)

corr_A = {}
corr_B = {}
corr_AB_full = {}
corr_A_B_full = {}
corrs = ( corr_A, corr_B, corr_AB_full, corr_A_B_full )

In [2]:
N = 10**4 # number of spins
S = N/2
order_cap = 35 # order limit for short-time correlator expansion

fig_dir = "../figures/"

time_steps = 10**3 # time steps in plot
ivp_tolerance = 1e-10 # relative error tolerance in numerical integrator
max_tau = 2 # for simulation: chi * max_time = max_tau * N **(-2/3)

# determine simulation times in units of the OAT strength \chi
tau_vals = np.linspace(0, max_tau, time_steps)
times = tau_vals * N**(-2/3)

NAT, OAT, TAT, TNT, TNTB = "NAT", "OAT", "TAT", "TNT", "TNTB"
methods = [ NAT, OAT, TAT, TNT, TNTB ]
dicke = "dicke"

init_state = (0,1,0)
h_zxy = {}
h_zxy[NAT] = {}
h_zxy[OAT] = { (2,0,0) : 1 }
h_zxy[TAT] = { (2,0,0) : +1/3,
               (0,0,2) : -1/3 }
h_zxy[TNT] = { (2,0,0) : 1,
               (0,1,0) : S }
h_zxy[TNTB] = { (2,0,0) : 1,
                (0,1,0) : -S }

dec_mat = None
collective_dec = 1/2
single_dec = collective_dec * N
no_dec = None
dec_p = (single_dec,0,0)
dec_z = (0,single_dec,0)
dec_m = (0,0,single_dec)
dec_P = ( (0,0,0), (collective_dec,0,0) )
dec_Z = ( (0,0,0), (0,collective_dec,0) )
dec_M = ( (0,0,0), (0,0,collective_dec) )
dec_rates = [ no_dec, dec_m, dec_M, dec_p, dec_P, dec_z, dec_Z ]
titles = { no_dec : "No decoherence",
           dec_p : r"$\gamma_+=N\chi/2$",
           dec_P : r"$\Gamma_+=\chi/2$",
           dec_m : r"$\gamma_-=N\chi/2$",
           dec_M : r"$\Gamma_-=\chi/2$",
           dec_z : r"$\gamma_{\mathrm{z}}=N\chi/2$",
           dec_Z : r"$\Gamma_{\mathrm{z}}=\chi/2$" }
titles[dicke] = titles[no_dec]

A_zxy = { (0,1,0) : 1,
          (0,0,1) : 1j }
B_zxy = { (0,1,0) : 1,
          (0,0,1) : -1j }
symbol_A = "Sp"
symbol_B = "Sm"

In [3]:
##########################################################################################
# Dicke manifold simulation methods
##########################################################################################

II = scipy.sparse.identity(N+1)
S_op_vec, _ = spin_op_vec_mat_dicke(N)

def vec_to_dicke(op_vec_zxy):
    op_dicke = 0 * II
    for op, val in op_vec_zxy.items():
        op_dicke_term = II.copy()
        for mm in range(3):
            for ii in range(op[mm]):
                op_dicke_term = op_dicke_term @ S_op_vec[mm]
        op_dicke += val * op_dicke_term
    return op_dicke

A_dicke = vec_to_dicke(A_zxy)
B_dicke = vec_to_dicke(B_zxy)
AB_dicke = A_dicke @ B_dicke
BA_dicke = B_dicke @ A_dicke
H = { method : vec_to_dicke(h_zxy[method]) for method in methods }

init_state_dicke = coherent_spin_state(init_state, N)
def get_states(hamiltonian, extra_op = II):
    return solve_ivp(lambda time, state : -1j * ( hamiltonian @ state ),
                     (0,times[-1]), extra_op @ init_state_dicke, t_eval = times,
                     rtol = ivp_tolerance, atol = ivp_tolerance).y

def corr_vals(op_mat, hamiltonian, prepend_op = II, append_op = II):
    kets = get_states(hamiltonian, append_op)
    if (prepend_op != append_op).nnz == 0: # if prepend_op == append_op
        bras = kets
    else:
        bras = get_states(hamiltonian, prepend_op.conj().T)
    return np.array([ bras[:,tt].conj() @ ( op_mat @ kets[:,tt] )
                      for tt in range(times.size) ])

##########################################################################################
# full simulation methods
##########################################################################################

if N < 10**4:
    A_pzm = vec_zxy_to_pzm(A_zxy)
    B_pzm = vec_zxy_to_pzm(B_zxy)
    h_pzm = { method : vec_zxy_to_pzm(h_zxy[method])
              for method in methods }
else:
    A_pzm = vec_zxy_to_pzm(A_zxy, init_state)
    B_pzm = vec_zxy_to_pzm(B_zxy, init_state)
    h_pzm = { method : vec_zxy_to_pzm(h_zxy[method], init_state)
              for method in methods }
    init_state = (-1,0,0)

AB_pzm = multiply_vecs(A_pzm, B_pzm)
BA_pzm = multiply_vecs(B_pzm, A_pzm)

def get_corrs(corr_op, h_pzm, dec_rate, prepend_op = None, append_op = None):
    if dec_rate is not None and len(dec_rate) == 3:
        dec_rate = [ dec_rate, (0,0,0) ]
    correlators = compute_correlators(times, order_cap, N, init_state, h_pzm,
                                      dec_rate, dec_mat, corr_op.keys(),
                                      prepend_op, append_op)
    return sum( val * correlators[op] for op, val in corr_op.items() )

In [4]:
if N < 10**3:
     from time import time
     start = time()
     for corr in corrs: corr[dicke] = {}
     for method in methods:
         print(method)
         corr_A[dicke][method] = corr_vals(A_dicke, H[method])
         corr_B[dicke][method] = corr_vals(B_dicke, H[method])
         corr_AB_full[dicke][method] = corr_vals(AB_dicke, H[method])
         corr_A_B_full[dicke][method] = corr_vals(A_dicke, H[method], append_op = B_dicke)
     print(time()-start)
else:
    for corr in corrs:
        try: del corr[dicke]
        except: None

In [5]:
from time import time
start = time()
for dec_rate in dec_rates:
    for corr in corrs: corr[dec_rate] = {}
    print(dec_rate)
    for method in methods:
        print("",method)
        corr_A[dec_rate][method] = get_corrs(A_pzm, h_pzm[method], dec_rate)
        corr_B[dec_rate][method] = get_corrs(B_pzm, h_pzm[method], dec_rate)
        corr_AB_full[dec_rate][method] = get_corrs(AB_pzm, h_pzm[method], dec_rate)
        corr_A_B_full[dec_rate][method] = get_corrs(A_pzm, h_pzm[method], dec_rate, None, B_pzm)
print(time()-start)

None
 NAT


 OAT


 TAT


 TNT


 TNTB


...

((0, 0, 0), (0, 0.5, 0))
 NAT
 OAT


 TAT


 TNT


 TNTB


641.3538422584534


In [6]:
corr_AB = {}
corr_A_B = {}
for key in corr_A.keys():
    corr_AB[key] = {}
    corr_A_B[key] = {}
    for method in methods:
        corr_AB[key][method] \
            = corr_AB_full[key][method] - corr_A[key][method] * corr_B[key][method]
        corr_A_B[key][method] \
            = corr_A_B_full[key][method] - corr_A[key][method] * corr_B[key][method][0]

In [11]:
%matplotlib agg
max_plot_tau = 0.07 * max_tau
plot_idx = tau_vals <= max_plot_tau
figsize = (7,4)
params = { "text.usetex" : True,
           "font.size" : 8 }
plt.rcParams.update(params)

plot_methods = [ OAT, TAT, TNT ]
colors = [ "#4E79A7", "#F28E2B", "#E15759", "#76B7B2",
           "#59A14E", "#EDC949", "#B07AA2", "#FF9DA7",
           "#9C755F", "#BAB0AC" ]

grid_map = { no_dec : (0,0), dicke: (1,0) }
for col, dec_rate in enumerate(dec_rates[1::2]):
    grid_map[dec_rate] = (0,col+1)
for col, dec_rate in enumerate(dec_rates[2::2]):
    grid_map[dec_rate] = (1,col+1)

corrs = corr_A_B
corr_label = "C(t)"
filename = "two_time.pdf"

legend_y = 0.08
legend_x = 0.02
offset_x = 0.135
ylims = (-1.2,1.2)

##########################################################################################
plt.figure(figsize = figsize)
plot_dicke = dicke in corrs.keys()
rows, columns = 4, (len(corrs)+1) // 2

axes = {}
top_cell = 1 + ( 0 if plot_dicke else columns )
cells = (top_cell, top_cell+columns)
axes[no_dec] = plt.subplot(rows, columns, cells)

remaining_keys = [ key for key in corrs.keys() if key != no_dec ]
for key in remaining_keys:
    mm, nn = grid_map[key]
    top_cell = 2*columns*mm + nn + 1
    cells = (top_cell, top_cell+columns)
    axes[key] = plt.subplot(rows, columns, cells,
                            sharex = axes[no_dec], sharey = axes[no_dec])

for key, corr in corrs.items():
    axis = axes[key]

    for method, color in zip(plot_methods, colors):
        axis.plot(N*times[plot_idx], np.real(corr[method][plot_idx])/S, color = color)
        axis.plot(N*times[plot_idx], np.imag(corr[method][plot_idx])/S, color = color,
                  linestyle = "--")

    if grid_map[key][0] == 1 or ( key == no_dec and not plot_dicke ):
        axis.ticklabel_format(axis = "x", style = "scientific", scilimits = (0,2))
        axis.set_xlabel(r"Time ($N\chi t$)")
    else:
        plt.setp(axis.get_xticklabels(), visible = False)

    if grid_map[key][1] == 0:
        axis.ticklabel_format(axis = "y", style = "scientific", scilimits = (0,2))
    else:
        plt.setp(axis.get_yticklabels(), visible = False)
        axis.yaxis.get_offset_text().set_visible(False)

    axis.set_ylim(*ylims)
    axis.tick_params(direction = "in")
    axis.grid(True)

for key, axis in axes.items():
    axis.set_title(titles[key])

linestyle_handles = [ Line2D([0], [0], color = "k", linestyle = "-"),
                      Line2D([0], [0], color = "k", linestyle = "--", dashes = (3,1)) ]
linestyle_labels = [ r"$\mathrm{Re}[C(t)]$", r"$\mathrm{Im}[C(t)]$" ]
plt.gcf().legend(linestyle_handles, linestyle_labels,
                 loc = "lower left", bbox_to_anchor = (legend_x,legend_y))

color_handles = [ Line2D([0], [0], color = color) for color in colors ]
plt.gcf().legend(color_handles, plot_methods,
                 loc = "lower left", bbox_to_anchor = (legend_x+offset_x,legend_y))

plt.tight_layout()
plt.savefig(fig_dir + filename)